In [2]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\neko\onedrive\desktop\learning\homework\mlops_zoomcamp\venv\lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: mlflow


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
y_pred.std()

5.28140357655334

In [11]:
df['ride_id'] = f'{2022:04d}/{2:02d}_' + df.index.astype('str')

In [24]:
df["predictions"] = y_pred

In [26]:
df_result = df[["ride_id", "predictions"]]

In [27]:
df_result.head()

,ride_id,predictions
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


In [28]:
df_result.to_parquet(
    "feb-2022.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)